In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkNlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [2]:
spark

In [3]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from functools import reduce
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import pandas as pd
import re
import string

# 2017

In [4]:
cong_2017 = spark.read.csv("s3://502finalprojbucky/YearlyCongress/cong2017.csv",header=False)

In [5]:
cong_2017.head(1)

[Row(_c0='0', _c1='940593982404530177', _c2='RepBarragan', _c3='816833925456789505', _c4='2017-12-12T09:47:35-05:00', _c5='https://www.twitter.com/RepBarragan/statuses/940593982404530177', _c6='Instead of holding polluters accountable and enforcing laws that prevent cancer and asthma, Scott Pruitt is working to slash EPA’s public health protection budgets. Let’s remind @EPAScottPruitt that he works on behalf of the American people, not oil and gas industry executives. https://twitter.com/nytimes/status/940419693529182214 QT @nytimes The EPA under President Trump has slowed down efforts to punish polluters and has even curbed the powers of its enforcement officers http://nyti.ms/2ygsDhH', _c7='Twitter Web Client')]

In [6]:
cong_2017.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hadoop/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [9]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('rt')
eng_stopwords.append('qt')
eng_stopwords.append('&amp')
eng_stopwords.append('amp')
eng_stopwords.append('+')
eng_stopwords.append('w')
eng_stopwords.append('today')
eng_stopwords.append('live')
eng_stopwords.append('make')
eng_stopwords.append('hear')
eng_stopwords.append('meet')
eng_stopwords.append('thank')
eng_stopwords.append('see')
eng_stopwords.append('time')
eng_stopwords.append('day')
eng_stopwords.append('watch')
eng_stopwords.append('get')
eng_stopwords.append('th')
eng_stopwords.append('year')
eng_stopwords.append('la')
eng_stopwords.append('pm')
eng_stopwords.append('hr')
eng_stopwords.append('rep')
eng_stopwords.append('come')
eng_stopwords.append('last')
eng_stopwords.append('dc')

In [10]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.ml import Pipeline
import pyspark.sql.functions as f
import boto3, os, datetime

In [18]:
##### _c5 is the tweet
documentAssembler = DocumentAssembler() \
    .setInputCol('_c6') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [19]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [20]:
data = cong_2017.filter(cong_2017['_c6'].isNull() == False)

In [21]:
cong_2017_token = pipeline.fit(data).transform(data)

In [22]:
cong_2017_token.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 'document',
 'token',
 'normalized',
 'lemma',
 'clean_lemma',
 'finished_clean_lemma']

In [31]:
cong_2017_token.select('finished_clean_lemma').show(20)

+--------------------+
|finished_clean_lemma|
+--------------------+
|[instead, hold, p...|
|[join, senbillnel...|
|[honored, able, j...|
|[goptaxplan, bill...|
|[happy, hanukkah,...|
|[grateful, calfir...|
|[realdonaldtrump,...|
|[senate, judiciar...|
|[waysandmeansgop,...|
|[speakerryan, pro...|
|[potus, sign, nda...|
|[tune, et, dwwg, ...|
|[happy, first, ha...|
|[cannot, good, co...|
|[right, im, offer...|
|[incredible, mome...|
|[thursday, republ...|
|[chadhgriffin, to...|
|[alexhousethomas,...|
|[potus, sign, law...|
+--------------------+
only showing top 20 rows



In [32]:
# expand the "finished_clean_lemma" column so that the words are not in a list
from pyspark.sql.functions import explode, col

all_words = cong_2017_token.withColumn("exploded_text", explode(col("finished_clean_lemma")))

In [33]:
all_words.select('exploded_text').show(5)

+-------------+
|exploded_text|
+-------------+
|      instead|
|         hold|
|     polluter|
|  accountable|
|      enforce|
+-------------+
only showing top 5 rows



In [34]:
counts = all_words.groupby('exploded_text').count()

In [35]:
counts

DataFrame[exploded_text: string, count: bigint]

In [36]:
counts.show(5)

+-------------+-----+
|exploded_text|count|
+-------------+-----+
|      calfire|  443|
|         hope| 1882|
|        still| 2611|
|     medicare| 1647|
|       travel|  599|
+-------------+-----+
only showing top 5 rows



In [37]:
from pyspark.sql.functions import *
counts_pd = counts.orderBy(desc('count')).toPandas()

In [38]:
counts_pd

,exploded_text,count
0,twitter,121092
1,web,62675
2,client,62617
3,iphone,49680
4,tax,33375
...,...,...
208887,httppoliticoyxcac,1
208888,httpwwwbreitbartcombigjournalismwikileaksjourn...,1
208889,httppbstwimgcommediadpqbacwaeahbjpg,1
208890,httpmprnoticiascomgonzalezanunciaseguiraradarp...,1


In [ ]:
counts_pd.shape

In [ ]:
counts_pd2 = counts_pd.head(30)

In [ ]:
counts_pd2

In [ ]:
ax = counts_pd2.plot.barh(x='exploded_text', y='count', rot=0)

In [ ]:
counts_pd3 = counts_pd.head(100)
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = counts_pd3.exploded_text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

# 2018

In [ ]:
cong_2018 = spark.read.csv("s3://502finalprojbucky/YearlyCongress/cong2018.csv",header=False)
data = cong_2018.filter(cong_2018['_c5'].isNull() == False)
cong_2018_token = pipeline.fit(data).transform(data)
all_words = cong_2018_token.withColumn("exploded_text", explode(col("finished_clean_lemma")))
counts = all_words.groupby('exploded_text').count()
counts_pd = counts.orderBy(desc('count')).toPandas()
counts_pd2 = counts_pd.head(30)

In [ ]:
ax = counts_pd2.plot.barh(x='exploded_text', y='count', rot=0)

In [ ]:
counts_pd3 = counts_pd.head(100)
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = counts_pd3.exploded_text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
cong_2018.select("_c5").show(5)

# 2019

In [ ]:
cong_2019 = spark.read.csv("s3://502finalprojbucky/YearlyCongress/cong2018.csv",header=False)
data = cong_2019.filter(cong_2019['_c5'].isNull() == False)
cong_2019_token = pipeline.fit(data).transform(data)
all_words = cong_2018_token.withColumn("exploded_text", explode(col("finished_clean_lemma")))
counts = all_words.groupby('exploded_text').count()
counts_pd = counts.orderBy(desc('count')).toPandas()
counts_pd2 = counts_pd.head(30)

In [ ]:
ax = counts_pd2.plot.barh(x='exploded_text', y='count', rot=0)

In [ ]:
counts_pd3 = counts_pd.head(100)
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = counts_pd3.exploded_text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()